# Load data from STL10 

In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision
import numpy as np
from Dataset import STL10_Relabel

%load_ext autoreload
%autoreload 2

%matplotlib inline

n = 4

dataset = STL10_Relabel()
print(dataset)
trainsubset = torch.utils.data.Subset(dataset, np.random.choice(len(dataset), n**2, replace=False))

# Visulize dataset

In [ ]:
from visualize import Datavisualize

T = torchvision.transforms.Compose([torchvision.transforms.ToPILImage(),torchvision.transforms.Grayscale(),torchvision.transforms.ToTensor()])
Datavisualize(n, trainsubset, title = 'Orginal', datapos = 1)

In [ ]:
Datavisualize(n, trainsubset, title = 'Grayscale', cmap = 'gray')

# Visulize Initial Model
Colorizer should output a gray image  
Discriminator should output a tensor with shape [1,2]

In [ ]:
from Dataset import RandomSet

testset = RandomSet()

In [ ]:
from model import Colorizer, Discriminator
from visualize import compare
mG = Colorizer()
mD = Discriminator()

compare(n, testset, mG)

In [ ]:
example = testset.__getitem__(0)[0]
example = mG(T(example).unsqueeze(0))
print(mD(example))

# Pretrain Colorizer with orginal image

In [ ]:
from Trainer import Trainer
tr = Trainer(mG, mD, dataset)
tr.pretrain(0)

## Now check how well the pretrian does

In [ ]:
mG.cpu()
compare(n, testset, mG)

# Use GAN to make colorizer output become more realistic

In [ ]:
G_loss, D_loss_real, D_loss_fake = tr.train(1)

In [ ]:
fig, ax = plt.subplots()
xG = np.arange(len(G_loss))
xD = np.arange(len(D_loss_real))*(len(G_loss)/len(D_loss_real))
ax.plot(xG, G_loss, label = 'G_loss')
ax.plot(xD, D_loss_real, label = 'D_loss_real')
ax.plot(xD, D_loss_fake, label = 'D_loss_fake')
ax.set_xlabel('times')
ax.set_ylabel('loss')
ax.legend()
plt.show()

## Now check if GAN make things looks better

In [ ]:
mG.cpu()
compare(n, testset, mG)